#### The data shall be saved in the Apache Parquet file format for ease of use and time efficiency.

This is a relatively complex solution to the problem I will take the easy route it should be easier to manage.

# Imports

In [1]:
from read_data import data_to_dataframe
import pandas as pd

In [2]:
import datetime
import pyarrow

# Pandas Options

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
pd.set_option('display.max_colwidth', None)

# Constructing the original DataFrame

In [5]:
data_directory = '/home/dragomy/Documents/Jupyter/oneWeek'
df = data_to_dataframe(data_directory)

# Flatten the original DataFrame

#### For the flattening we convert our df to the second normalform

In [6]:
basics = df[["sophoraId","title","date","type","breakingNews","ressort"]]

The basics collumn does not contain any collumns with nested datatypes

In [7]:
location = df[["sophoraId","geotags","regionId","regionIds"]]
location = location.explode('geotags').explode('regionIds')

location has several lists that need to be flattened luckily pandas has a function for that

In [8]:
links = df[["sophoraId","teaserImage","updateCheckUrl","details","detailsweb","shareURL"]]

def flatten_teaser_image(teaser_image):
    if isinstance(teaser_image, dict):
        flat_data = {
            'teaserImage_alttext': teaser_image.get('alttext', ''),
            'type': teaser_image.get('type', ''),
        }
        image_variants = teaser_image.get('imageVariants', {})
        if isinstance(image_variants, dict):
            for key, value in image_variants.items():
                flat_data[f'imageVariants_{key}'] = value
        return flat_data
    else:
        return {}

flattened_teaser_images = links['teaserImage'].apply(flatten_teaser_image)
flattened_df = pd.DataFrame(flattened_teaser_images.tolist())
links = pd.concat([links.drop(columns=['teaserImage']), flattened_df], axis=1)

links has a collumn containing a dictionary so this takes the keys and uses them as collumns

In [9]:
tags = df[["sophoraId","tags"]]
tags = tags.explode('tags')

def extract_tag(tmp_df):
    if isinstance(tmp_df, dict):  
        return tmp_df.get('tag')  
    else:
        return None  

tags['tags'] = tags['tags'].apply(extract_tag)

tags contains a list of dictionarys that always have tags as a key and the value is a string so i extract the string 

In [10]:
other = df[["sophoraId","tracking","topline","firstSentence","brandingImage","streams","alttext","copyright","comments"]]

def normalize_tracking_column(df):
    
    if 'tracking' in df.columns and df['tracking'].notna().any():
        
        tracking_df = df['tracking'].apply(lambda x: pd.Series(x[0]) if isinstance(x, list) and len(x) > 0 else pd.Series())
        tracking_df.columns = [f'tracking_{col}' for col in tracking_df.columns]
        df = pd.concat([df.drop(columns=['tracking']), tracking_df], axis=1)
    
    return df

other = normalize_tracking_column(other)

other contains the collumns i dont realy know what to do with

# Save as Parquet

In [11]:
def save_parquet(df, df_name):
    current_datetime = datetime.datetime.now()
    
    parquet_name = f"{df_name}_{current_datetime.strftime('%Y-%m-%d_%H-%M-%S')}.parquet.gzip"
    
    df.to_parquet(parquet_name, "pyarrow", "gzip")
    print(f"DataFrame saved as: {parquet_name}")

In [13]:
def save_all_to_parquet():
    save_parquet(basics, "basics")
    save_parquet(location, "location")
    save_parquet(links, "links")
    save_parquet(tags, "tags")
    save_parquet(other, "other")

In [14]:
save_all_to_parquet()

/tmp/ipykernel_37452/1865536185.py:6: FutureWarning: Starting with pandas version 3.0 all arguments of to_parquet except for the argument 'path' will be keyword-only.
  df.to_parquet(parquet_name, "pyarrow", "gzip")


DataFrame saved as: basics_2024-07-19_15-02-16.parquet.gzip
DataFrame saved as: location_2024-07-19_15-02-16.parquet.gzip
DataFrame saved as: links_2024-07-19_15-02-17.parquet.gzip
DataFrame saved as: tags_2024-07-19_15-02-19.parquet.gzip
DataFrame saved as: other_2024-07-19_15-02-20.parquet.gzip
